### Import Statements:

In [1]:
import urllib
import os
import brotli
import csv
import html
import time
import random
import json
import requests
from pathlib import PurePath
from collections import defaultdict
import pandas as pd


### Defining Functions:

In [2]:
def wait_random_time(min_time, max_time):
    """Waits for a random amount of time between min_time and max_time (inclusive)."""
    sleep_time = random.uniform(min_time, max_time)
    print(f"Waiting random amount of seconds ({sleep_time}s in this case) before continuing...")
    time.sleep(sleep_time)

In [45]:
def RequestAlbumsPage(page_number, song_dict_):
    wait_random_time(9,17)
    print(f"Requesting page {page_number} of pitchfork.com/reviews/albums ...")
    pitchfork_pg_x = r'https://pitchfork.com/reviews/albums/?page=' + f'{page_number}'
    req = urllib.request.Request(pitchfork_pg_x)
    r = urllib.request.urlopen(req).read()
    bebe_decoded = r.decode("utf-8")
    
    # list split up by album text code things:
    splitbythings = bebe_decoded.split('"@type":"ListItem","name":')
    
    clean_split_items = []

    for i in range(len(splitbythings)):
        if i == 0:
            pass
        else:
            itersplit_dirty = splitbythings[i]
            itersplit_1 = itersplit_dirty.split(',"position"')[0]
            if r"\u002Freviews\u002Falbums" not in itersplit_1:
                clean_split_items.append([itersplit_1, itersplit_dirty])

    for item in clean_split_items:
        clean = item[0]
        dirty = item[1]
        firstsplit = clean.split('"*')[1]
        albumname_rough_parse = firstsplit.split('","url":"')[0]
        secondsplit = firstsplit.split('","url":"')[1]
        url_parsed = secondsplit.split('/"')[0]

        if albumname_rough_parse[-1:] == '*':
             albumname_clean = albumname_rough_parse[:-1]
        elif albumname_rough_parse[-1:] == 'P':
            albumname_clean = albumname_rough_parse[:-4] + " <EP>"
        else:
            print("Woah buddy, you haven't accounted for this case!")

        url_purepath = PurePath(url_parsed)
        url_songdesc = url_purepath.name
        
        song_dict_[url_songdesc]['album_name'] = albumname_clean
        song_dict_[url_songdesc]['review_url'] = url_parsed
        
        
        song_split_str = url_songdesc + '"},"showAssetOnly"'
        artistname_split_1 = itersplit_dirty.split(song_split_str)
        artistname_split_2 = artistname_split_1[0].split('"subHed":{"name":"')[-1]
        artist_name_parsed = artistname_split_2.split('","url":')[0]
        
        song_dict_[url_songdesc]['artist_name'] = artist_name_parsed
 
    song_dict_reg = dict(song_dict_)
    for index, key_string in enumerate(song_dict_reg):
        individual_review_url = song_dict_reg[key_string]['review_url']
        print(f"Prepping to request {key_string} ({index+1}/{len(song_dict_reg)} - page {page_number})...")
        wait_random_time(9,21)
        req = urllib.request.Request(individual_review_url)
        r = urllib.request.urlopen(req).read()
        jeen_decoded = r.decode("utf-8")

        infoslicefields_split1 = jeen_decoded.split(',"infoSliceFields":')[1]
        infoslicefields_split2 = infoslicefields_split1.split(',"isMusicReview":')
        infoslicefields = json.loads(infoslicefields_split2[0])

        genre_parsed = infoslicefields['genre']
        song_dict_reg[key_string]['genre'] = genre_parsed

        try:
            label_parsed = infoslicefields['label']
            song_dict_reg[key_string]['label'] = label_parsed
        except:
            song_dict_reg[key_string]['label'] = ""

        reviewDate_parsed = infoslicefields['reviewDate']
        song_dict_reg[key_string]['review_date'] = reviewDate_parsed

        releaseYear_parsed = infoslicefields['releaseYear']
        song_dict_reg[key_string]['release_date'] = releaseYear_parsed

        nextpart_split1 = infoslicefields_split1.split('},"isMusicReview":')[1]
        nextpart_split2 = nextpart_split1.split(',"lede":{')[0]
        nextpart_split3 = nextpart_split2.split(',"isTrackReview":')
        isMusicReview_parsed = nextpart_split3[0]
        song_dict_reg[key_string]['is_music_review'] = isMusicReview_parsed

        nextpart_split4 = nextpart_split3[1].split(',"musicRating":')
        isTrackReview_parsed = nextpart_split4[0]
        song_dict_reg[key_string]['is_track_review'] = isTrackReview_parsed

        nextpart_split5 = nextpart_split4[1].split(',"modifiedDate":"')
        lildict = json.loads(nextpart_split5[0])
        isBestNewMusic_parsed = lildict['isBestNewMusic']
        song_dict_reg[key_string]['is_best_new_music'] = isBestNewMusic_parsed

        isBestNewReissue_parsed = lildict['isBestNewReissue']
        song_dict_reg[key_string]['is_best_new_reissue'] = isBestNewReissue_parsed

        score_parsed = lildict['score']
        song_dict_reg[key_string]['rating'] = score_parsed

        authorname_split1 = jeen_decoded.split('><meta name="author" content="')
        authorname_parsed = authorname_split1[1].split('"/><meta name=')[0]
        song_dict_reg[key_string]['review_author_name'] = authorname_parsed

        descrip_split1 = jeen_decoded.split('><meta name="description" content="')[1]
        descrip_parsed = descrip_split1.split('"/><meta name="id" content="')[0]
        song_dict_reg[key_string]['review_description'] = descrip_parsed

        content_id_split1 = jeen_decoded.split('><meta name="id" content="')[1]
        content_id_parsed = content_id_split1.split('"/><meta name="keywords" content="')[0]
        song_dict_reg[key_string]['content_id'] = content_id_parsed

        photo_id_split1 = jeen_decoded.split(r'https://media.pitchfork.com/photos/')[1]
        photo_id_parsed = photo_id_split1.split(r'/16:9/w_1000,c_limit/')[0]
        song_dict_reg[key_string]['photo_id'] = photo_id_parsed

        full_photo_url = r'https://media.pitchfork.com/photos/' + photo_id_parsed + r'/master/w_1280,c_limit'
        song_dict_reg[key_string]['full_photo_url'] = full_photo_url

    return song_dict_reg

# Running the script:

#### Create empty dictionary:

In [46]:
mydicto = {}
mydicto = defaultdict(dict)

#### Enter in page of pitchfork.com/reviews/album you will be scraping:

In [47]:
myreturn = RequestAlbumsPage(1, mydicto)

Waiting random amount of seconds (10.471471455025853s in this case) before continuing...
Requesting page 1 of pitchfork.com/reviews/albums ...
Prepping to request yungmorpheus-dirty-art-club-a-spyglass-to-ones-face (1/96 - page 1)...
Waiting random amount of seconds (12.901986479405902s in this case) before continuing...
Prepping to request gabriel-jacoby-gutta-child-ep (2/96 - page 1)...
Waiting random amount of seconds (11.787945813454257s in this case) before continuing...
Prepping to request akai-solo-no-control-no-glory (3/96 - page 1)...
Waiting random amount of seconds (13.499029183177331s in this case) before continuing...
Prepping to request sinn6r-federal (4/96 - page 1)...
Waiting random amount of seconds (15.131262787917912s in this case) before continuing...
Prepping to request fred-again-usb002 (5/96 - page 1)...
Waiting random amount of seconds (10.741188820664505s in this case) before continuing...
Prepping to request mavi-the-pilot (6/96 - page 1)...
Waiting random amo

Prepping to request the-hellp-riviera (52/96 - page 1)...
Waiting random amount of seconds (13.778377358186358s in this case) before continuing...
Prepping to request el-plvybxy-retrospective-frequencies (53/96 - page 1)...
Waiting random amount of seconds (13.61089966891329s in this case) before continuing...
Prepping to request summer-walker-finally-over-it (54/96 - page 1)...
Waiting random amount of seconds (9.250490822461387s in this case) before continuing...
Prepping to request wrens-half-of-what-you-see (55/96 - page 1)...
Waiting random amount of seconds (9.715100422033807s in this case) before continuing...
Prepping to request lade-laflair (56/96 - page 1)...
Waiting random amount of seconds (18.493542622395356s in this case) before continuing...
Prepping to request computer-station-on-the-hill (57/96 - page 1)...
Waiting random amount of seconds (14.349627725230796s in this case) before continuing...
Prepping to request juana-molina-doga (58/96 - page 1)...
Waiting random am

In [48]:
myreturn

{'yungmorpheus-dirty-art-club-a-spyglass-to-ones-face': {'album_name': 'A Spyglass to One’s Face',
  'review_url': 'https://pitchfork.com/reviews/albums/yungmorpheus-dirty-art-club-a-spyglass-to-ones-face',
  'artist_name': 'YUNGMORPHEUS \\u002F Dirty Art Club',
  'genre': 'Rap / Electronic',
  'label': 'Fat Beats',
  'review_date': 'December 18, 2025',
  'release_date': '2025',
  'is_music_review': 'true',
  'is_track_review': 'false',
  'is_best_new_music': False,
  'is_best_new_reissue': False,
  'rating': 7.7,
  'review_author_name': 'Dash Lewis',
  'review_description': 'Spurred on by Charlotte producer Dirty Art Club’s spacious, hypnotic beats, the L.A.-based rapper sounds more reflective than usual, but his unflappable chill still prevails.',
  'content_id': '694049ebb35c9460e500fd1f',
  'photo_id': '694048797a1aa927f9b25bf8',
  'full_photo_url': 'https://media.pitchfork.com/photos/694048797a1aa927f9b25bf8/master/w_1280,c_limit'},
 'gabriel-jacoby-gutta-child-ep': {'album_name':

#### Save returned dictionary to a csv (enter in csv name)

In [49]:
df = pd.DataFrame.from_dict(myreturn, orient='index')
df.index.name = 'Album_Key'

datafolder = r'C:\Users\michael.felzan\Desktop\Misc\Pitchforkle\Data'

output_csv = os.path.join(datafolder, "page1_12182025.csv")
df.to_csv(output_csv)

#### Cycle through returned dictionary of song info, request and save album thumbnails for each review:

In [53]:
for index, key_ in enumerate(myreturn):
    datafolder = r'C:\Users\michael.felzan\Desktop\Misc\Pitchforkle\Data'
    image_url = myreturn[key_]['full_photo_url']
    image_name = key_ + ".png"
    wait_random_time(9,21)
    img_data = requests.get(image_url).content
    image_output = os.path.join(datafolder, image_name)
    with open(image_output, 'wb') as handler:
        handler.write(img_data)
        print(f"saved {key_}.png ({index+1}/{len(myreturn)})")

Waiting random amount of seconds (19.482516334031512s in this case) before continuing...
saved yungmorpheus-dirty-art-club-a-spyglass-to-ones-face.png
Waiting random amount of seconds (14.657698985806412s in this case) before continuing...
saved gabriel-jacoby-gutta-child-ep.png
Waiting random amount of seconds (13.633526822573298s in this case) before continuing...
saved akai-solo-no-control-no-glory.png
Waiting random amount of seconds (11.395063462193045s in this case) before continuing...
saved sinn6r-federal.png
Waiting random amount of seconds (19.55301757974162s in this case) before continuing...
saved fred-again-usb002.png
Waiting random amount of seconds (20.20752946815544s in this case) before continuing...
saved mavi-the-pilot.png
Waiting random amount of seconds (18.68616689976376s in this case) before continuing...
saved jawnino-surf-gang-amnesia.png
Waiting random amount of seconds (12.541478868065486s in this case) before continuing...
saved dj-narciso-dentro-de-mim.png


saved oneohtrix-point-never-tranquilizer.png
Waiting random amount of seconds (19.5117571047873s in this case) before continuing...
saved the-belair-lip-bombs-again.png
Waiting random amount of seconds (13.172764753325788s in this case) before continuing...
saved wata-igarashi-my-supernova.png
Waiting random amount of seconds (15.224703580798547s in this case) before continuing...
saved aesop-rock-float.png
Waiting random amount of seconds (16.78570935242839s in this case) before continuing...
saved husker-du-1985-the-miracle-year.png
Waiting random amount of seconds (15.493459650558663s in this case) before continuing...
saved smerz-big-city-life-edits.png
Waiting random amount of seconds (13.486700492923612s in this case) before continuing...
saved sorry-cosplay.png
Waiting random amount of seconds (15.952513354275535s in this case) before continuing...
saved g-herbo-lil-herb.png
Waiting random amount of seconds (14.561284744679472s in this case) before continuing...
saved after-afte

### The following code is for debugging / testing new features!

In [39]:


individual_review_url = r'https://pitchfork.com/reviews/albums/yungmorpheus-dirty-art-club-a-spyglass-to-ones-face/'
req = urllib.request.Request(individual_review_url)
r = urllib.request.urlopen(req).read()
jeen_decoded = r.decode("utf-8")

infoslicefields_split1 = jeen_decoded.split(',"infoSliceFields":')[1]
infoslicefields_split2 = infoslicefields_split1.split(',"isMusicReview":')
infoslicefields = json.loads(infoslicefields_split2[0])

genre_parsed = infoslicefields['genre']
#song_dict_reg[key_string]['genre'] = genre_parsed

try:
    label_parsed = infoslicefields['label']
    #song_dict_reg[key_string]['label'] = label_parsed
except:
    pass

reviewDate_parsed = infoslicefields['reviewDate']
#song_dict_reg[key_string]['review_date'] = reviewDate_parsed

releaseYear_parsed = infoslicefields['releaseYear']
#song_dict_reg[key_string]['release_date'] = releaseYear_parsed

nextpart_split1 = infoslicefields_split1.split('},"isMusicReview":')[1]
nextpart_split2 = nextpart_split1.split(',"lede":{')[0]
nextpart_split3 = nextpart_split2.split(',"isTrackReview":')
isMusicReview_parsed = nextpart_split3[0]
#song_dict_reg[key_string]['is_music_review'] = isMusicReview_parsed

nextpart_split4 = nextpart_split3[1].split(',"musicRating":')
isTrackReview_parsed = nextpart_split4[0]
#song_dict_reg[key_string]['is_track_review'] = isTrackReview_parsed

nextpart_split5 = nextpart_split4[1].split(',"modifiedDate":"')
lildict = json.loads(nextpart_split5[0])
isBestNewMusic_parsed = lildict['isBestNewMusic']
#song_dict_reg[key_string]['is_best_new_music'] = isBestNewMusic_parsed

isBestNewReissue_parsed = lildict['isBestNewReissue']
#song_dict_reg[key_string]['is_best_new_reissue'] = isBestNewReissue_parsed

score_parsed = lildict['score']
#song_dict_reg[key_string]['rating'] = score_parsed

authorname_split1 = jeen_decoded.split('><meta name="author" content="')
authorname_parsed = authorname_split1[1].split('"/><meta name=')[0]
#song_dict_reg[key_string]['review_author_name'] = authorname_parsed

descrip_split1 = jeen_decoded.split('><meta name="description" content="')[1]
descrip_parsed = descrip_split1.split('"/><meta name="id" content="')[0]
#song_dict_reg[key_string]['review_description'] = descrip_parsed

content_id_split1 = jeen_decoded.split('><meta name="id" content="')[1]
content_id_parsed = content_id_split1.split('"/><meta name="keywords" content="')[0]
#song_dict_reg[key_string]['content_id'] = content_id_parsed

photo_id_split1 = jeen_decoded.split(r'https://media.pitchfork.com/photos/')[1]
photo_id_parsed = photo_id_split1.split(r'/16:9/w_1000,c_limit/')[0]
#song_dict_reg[key_string]['photo_id'] = photo_id_parsed

full_photo_url = r'https://media.pitchfork.com/photos/' + photo_id_parsed + r'/master/w_1280,c_limit'
#song_dict_reg[key_string]['full_photo_url'] = full_photo_url

In [ ]:
waybackmachine_url = r'https://web.archive.org/web/20160310042721/http://pitchfork.com/reviews/albums/3/'

req = urllib.request.Request(waybackmachine_url)
r = urllib.request.urlopen(req).read()
peen_decoded = r.decode("utf-8")

peen_decoded